<a href="https://colab.research.google.com/github/GuilhermePelegrina/Mackenzie/blob/main/Aulas/2s2024/TIC/Aula_08_Regressao_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/logo_mackenzie.png'>


# **Problemas de Regressão**

Problemas de regressão consistem em modelar uma relação (linear ou não-linear) entre uma variável dependente (variável de saída) e uma ou mais variáveis independentes (variáveis de entrada). O que difere problemas de regressão para os problemas de classificação é que, agora, buscamos prever um valor contínuo (e não valores 0 ou 1 representando classes).

Os modelos de regressão têm uma ampla gama de aplicações em diversas áreas, tais como:

- Estudo da relação entre a renda e a educação.
- Análise do impacto de tratamentos médicos em resultados de pacientes.
- Estimativa de vendas com base em despesas com publicidade.
- Análise da relação entre velocidade e consumo de combustível em veículos.
- Previsão de níveis de água em rios com base em dados meteorológicos.

# **Regressão Linear**

Em estatística, Regressão Linear é uma equação que se usa para estimar o valor esperado de uma variável, $y$ (chamada variável dependente), a partir de uma função linear dos valores de outras variáveis $x_1, x_2,...,x_m$ (as variáveis independentes).

Antes de utilizar um modelo de regressão linear, é importante avaliar a relação linear entre as variáveis de interesse. Para isso, é possível empregar gráficos de dispersão (*scatterplots*) e calcular o coeficiente de correlação.

## Exemplo - Sales/Marketing

Para aprender os conceitos desta aula de Regressão Linear, vamos usar o conjunto de dados [Advertising](https://www.kaggle.com/datasets/ashydv/advertising-dataset). Nesse conjunto de dados, o objetivo consiste em estimar o número de vendas (*Sales*) com base no orçamento destinado à publicidade para três mídias: *TV*, *Radio* e *Newspaper*.

In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Datasets/data_advertising.csv")
df.head()

Como uma primeira análise, podemos visualizar a relação entre as variáveis através de gráficos de dispersão.

**Lembre-se**: Gráficos de disperssão (*scatterplot*) são ferramentas úteis na análise de dados para visualizar a relação entre duas variáveis.

In [ ]:
# Gráficos de dispersão

import seaborn as sns
import matplotlib.pyplot as plt
sns.pairplot(df,
             x_vars=['TV', 'Radio', 'Newspaper'],
             y_vars='Sales',
             kind='scatter')
plt.show()

Além dos gráficos, podemos calcular o coeficiente de correlação entre as variáveis.

**Lembre-se**: O coeficiente de correlação ($r$) mede a força linear entre as variáveis independentes e dependentes. Seu valor varia de -1 a 1, de tal forma que:

- Um valor de $r$ próximo a 1 indica que à medida que uma variável aumenta, a outra também tende a aumentar.
- Um valor de $r$ próximo a -1 indica que à medida que uma variável aumenta, a outra tende a diminuir.
- Um valor de $r$ próximo a 0 indica uma correlação linear fraca ou inexistente entre as variáveis.

In [ ]:
# Correlação linear

df.corr()[["Sales"]]

Tanto visualmente quanto verificado pelo coeficiente de correlação, podemos assumir que a variável *Sales* está relacionada linearmente com a variável *TV*. Por outro lado, *Radio* e *Newspaper* não possui uma relação tão forte com *Sales*.

Provavelmente, em um moelo de Regressão Linear, o uso da variável *TV* teria um impacto maior que as demais variáveis ao predizer o número de vendas. Vamos analisar essas hipóteses implementando o modelo de Regressão Linear.

## Regressão Linear Simples

A Regressão Linear Simples permite modelar a relação entre duas variáveis, uma sendo a variável resposta $y$ e a outra sendo preditora ou explicativa. Essa técnica assume uma relação linear entre as variáveis, a qual pode ser aproximada pela seguinte equação:

$$y_i=\beta_0 + \beta_1 x_i + ϵ_i,$$

em que $i = 1, \cdots, n$ indica uma amostra no conjunto de $n$ dados, $y_i$ é a variável que tentamos prever, $x_i$ é variável explicativa, $\beta_0$ é o intercepto; $\beta_1$ é o coeficiente angular e $\epsilon$ o erro.

<img src='https://raw.githubusercontent.com/guilhermepelegrina/Mackenzie/main/Aulas/Figuras/fig_regr_linear.png'>



Note que para criar a reta de regressão é preciso considerar todos os dados (ou amostras $i = 1, \cdots, n$) disponíveis para `treinar` o modelo. E, para o conjunto das amostras, o objetivo é de encontrar os valores ideais para os parâmetros $\beta_0$ e $\beta_1$ de tal forma que o erro total seja minimizado.

Note que a simples soma dos erros, isto é, $\sum_{i=1}^{n} \epsilon_i$, pode resultar em um valor próximo de zero. Isso porque os erros $\epsilon_i$ podem ser tanto positivos quanto negativos, e na soma um erro positivo "pode compensar" um erro negativo. Sendo assim, um método que pode ser adotado para encontrar os valores ótimos de $\beta_0$ e $\beta_1$, chamado de Mínimos Quadrados Ordinários (MQO) ou OLS (do inglês Ordinary Least Squares), consiste em resolver o seguinte problema de otimização:

$$OLS(\beta_0, \beta_1)=\min_{\beta_0, \beta_1} \sum_{i=1}^{n} \epsilon^2_i = \min_{\beta_0, \beta_1} \sum_{i=1}^{n}(y_i -\beta_0 - \beta_1 x_i )^2.$$

em que $\epsilon_i$ é a diferença entre os valores observados ($y_i$) e previstos a partir do modelo linear ($\beta_0 + \beta_1 x_i$).

A solução desse problema de otimização é bem conhecida. A partir das derivadas parciais em relação a $\beta_0$ e $\beta_1$ utilizando a regra da cadeia e igualando ambas expressões à zero, podemos encontrar os valores de $\beta_0$ e $\beta_1$ que minimizam $OLS(\beta_0, \beta_1)$, isto é, a soma do quadrados dos erros.

Nesta aula, usaremos a biblioteca `statsmodels` para estimar esses parâmetros.

**Ajustando os parâmetros**

Vamos ajustar um modelo de regressão para analisar as vendas (*Sales*) utilizando apenas *TV* como variável preditora.

In [ ]:
# Fit the resgression line using 'OLS'

import statsmodels.formula.api as sm

model = sm.ols(formula='Sales ~ TV', data=df).fit()
print(model.summary())

O modelo de regressão obtido foi o seguinte:

$$\widehat{y} = 6.9748 + 0.055TV, $$

em que $\widehat{y}$ é a venda (`Sales`) esperada quando é investido um certo dinheiro de marketing em TV.

- Note que o valor-p (P>|t|) para a variável `TV` é 0.000 ($<$0.05). Isso indica que o orçamento destinado à publicidade em TV influencia, estatísticamente, nas vendas.

- Note que o `R-squared` é de 0.812. O que isso significa?

*Relembrando*:

- O `valor-p` associado ao coeficiente testa a hipótese nula de que tal coeficiente é igual a zero, ou seja, que o mesmo não tem efeito na variável predita. Então, um valor-p baixo (geralmente, < 0,05) indica que podemos rejeitar a hipótese nula. Isso significa que uma variável preditora que possui um valor-p baixo provavelmente terá uma contribuição significativa no modelo, uma vez que alterações no valor dessa variável preditora estão relacionadas a alterações na variável predita.

- `O coeficiente de determinação` (`R-squared`, ou $R^2$) é uma medida que indica o quanto a variabilidade da variável dependente é explicada pelas variáveis independentes em um modelo de regressão linear. O $R^2$ varia entre 0 e 1.

**Valores previstos pelo modelo de regressão!**

Como construir a reta de regressão?
Podemos calcular os valores preditos usando o modelo $6.9748+ 0.055TV$ substituindo cada valor de $x$ pelo valor observado.

In [ ]:
df['predicted'] = model.predict(df.TV)
df.head()

In [ ]:
# Scatterplot com Reta de Regressão

plt.scatter(df.TV, df.Sales, color='blue', label='Dados Observados')
plt.plot(df.TV, df.predicted, color='red', label='Reta de Regressão')

plt.xlabel('TV')
plt.ylabel('Sales')
plt.title('Scatterplot com Reta de Regressão')
plt.legend()
plt.show()

**Prevendo um novo valor pelo modelo de regressão!**

Usando o modelo de regressão linear ajustado, qual é o valor de vendas (`Sales`) esperado quando o orçamento em TV é de 153?

In [ ]:
# Prevendo para novos valores (não estão na base de dados)

new_value=pd.DataFrame({'TV': [153]})

model.predict(new_value)

In [ ]:
# Obtendo os intervalos de confiança de 95% para as previsões

model.get_prediction(new_value).conf_int()

**Interpretando os parâmetros**

A interpretação dos parâmetros da Regressão Linear é bem fácil. Basicamente, a alteração no valor predito com o aumento em uma unidade na variável preditora é equivalente ao parâmetro associado a tal variável. No exemplo das vendas, se ao invés de 153 como orçamento para TV tivéssemos 154, o aumento no número de vendas teria um incremento de 0,055 (o valor de $\beta$ que multiplica o orçamento em TV).

Verifique o resultado com esse novo dado.

In [ ]:
# Interpretando os parâmetros

new_value=pd.DataFrame({'TV': [154]})

model.predict(new_value)

**Análise de resíduos**

Um pressuposto que geralmente deve ser verificado nos modelos de regressão é que os resíduos devem ser aleatórios com média zero e seguir aproximadamente uma distribuição Normal. Quando essas suposições são satisfeitas, isso sugere que o modelo de regressão linear é apropriado para descrever a relação entre as variáveis.

In [ ]:
# Resíduos

residuals = model.resid_pearson

In [ ]:
# Gráfico de dispersão dos resíduos

plt.figure(figsize=(10, 3))

plt.subplot(1, 2, 1)
plt.scatter(df["predicted"], residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Valores ajustados')
plt.ylabel('Resíduos')
plt.title('Gráfico de Dispersão dos Resíduos')

plt.subplot(1, 2, 2)
plt.hist(residuals)

plt.tight_layout()
plt.show()

Esses gráficos sugerem que os resíduos se desenvolvem aleatoriamente em torno de zero e seguem aproximadamente uma distribuição Normal.

## Regressão Linear Múltipla

Um modelo de regressão múltipla é uma extensão do modelo de regressão linear simples, onde mais de uma variável independente (preditora) é usada para prever uma variável dependente (resposta).

$$y_i=\beta_0 + \beta_1 x_{1,i} + \beta_2 x_{2,i} + ...+ \beta_r x_{r,i}+ \epsilon_i,$$

em que $x_{1,i},...,x_{r,i}$ são as variáveis explicativas. Os parâmetros podem ser estimados usando OLS.

Vamos, agora, aplicar a Regressão Linear Múltipla no conjunto de dados *Advertising*, considerando todos os atributos no modelo.

In [ ]:
# Fit the resgression line using 'OLS'

model = sm.ols(formula='Sales ~ TV + Radio + Newspaper',
               data=df).fit()
print(model.summary())

O modelo de regressão obtido foi o seguinte:

$$\widehat{y} = 4.6251 + 0.0544TV + 0.1070Radio + 0.0003Newspaper, $$

- Note que a variável *Newspaper* não é significante (valor-p > 0.05).

- Note também que o `R-square` aumentou para 0.903.

**Valores previstos pelo modelo de regressão!**

Como no caso da regressão simples, podemos calcular os valores preditos usando o modelo $4.6251 + 0.0544TV + 0.1070Radio + 0.0003Newspaper$ substituindo cada valor das variáveis pelo valor observado.

In [ ]:
df['predicted'] = model.predict(df[["TV","Radio","Newspaper"]])
df.head()

**Prevendo um novo valor pelo modelo de regressão!**

Usando o modelo de regressão linear ajustado, qual é o valor de vendas (`Sales`) esperado quando o orçamento em TV é de 153, em Rádio é de 42 e em Jornal é de 59?

In [ ]:
# Prevendo para novos valores (não estão na base de dados)

new_values=pd.DataFrame({'TV': [153],
                       'Radio': [42],
                       'Newspaper': [59]})

model.predict(new_values)

In [ ]:
# Obtendo os intervalos de confiança de 95% para as previsões

model.get_prediction(new_values).conf_int()

**Análise de resíduos**

In [ ]:
# Resíduos

residuals = model.resid_pearson

In [ ]:
# Gráfico de dispersão dos resíduos

plt.figure(figsize=(10, 3))

plt.subplot(1, 2, 1)
plt.scatter(df["predicted"], residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Valores ajustados')
plt.ylabel('Resíduos')
plt.title('Gráfico de Dispersão dos Resíduos')

plt.subplot(1, 2, 2)
plt.hist(residuals)

plt.tight_layout()
plt.show()

Esses gráficos sugerem que os resíduos se desenvolvem aleatoriamente em torno de zero e seguem aproximadamente uma distribuição Normal.

**Interpretação da contribuição de cada atributo na regressão linear**

Vimos que os coeficientes da reta de regressão nos indica o impacto na predição com o aumento de uma unidade na variável associada. Mas, também podemos interpretar o quanto cada atributo contribuiu no modelo de regressão. Ou seja, podemos ter uma ideia de quais atributos são mais importantes no modelo e quais não tem tanta utilidade de serem usados.

Para essa interpretação, olhamos a coluna da estatística $t$ apresentada no quadro do OLS. O valor dessa estatística é medida como

$$t_{\beta_j} = \frac{\beta_j}{\text{std err}},$$

onde a coluna $\text{std err}$ indica o desvio padrão do valor do coeficiente na estimativa da reta de regressão. Note que, quanto maior o $\beta_j$ (pensando em um caso onde os atributos estão na mesma dimensão), maior a estatística $t$ e, portanto, maior o impacto de tal atributo na qualidade do modelo de regressão ajustado. Além disso, repare no $\text{std error}$. Um valor baixo desse desvio indica que a incerteza no coeficiente ajustado é menor, pois a variação em torno desse valor é menor. Portanto, sendo essa incerteza menor, maior vai set a estatística $t$ e, portanto, maior a significância do mesmo no modelo.

No exemplo mencionado anteriormente, *TV* tem o maior impacto no modelo, sendo o *Newspaper* o atributo de menor importância. Veja abaixo um cálculo e visualização das contribuições relativas de cada atributo.

In [ ]:
import matplotlib.pyplot as plt

std_error_dict = model.bse.to_dict() # Extrai os nomes dos atributos e std err e coloca em um dicionário
coeff_dict = model.params.to_dict() # Extrai os nomes dos atributos e coeficientes e coloca em um dicionário
t_names = []
t_val = []
for key in std_error_dict: # Separa os nomes e valores do dicionário
  if key != 'Intercept':
    t_names.append(key)
    t_val.append(abs(coeff_dict[key]/std_error_dict[key]))

plt.pie(t_val, labels=std_names, autopct='%0.2f%%') # Gráfico de pizza

Muito provavelmente, a remoção dele não impactaria na qualidade da reta de regressão. Vamos testar o modelo sem esse atributo e verificar a qualidade do mesmo.

In [ ]:
# Fit the resgression line using 'OLS'

model = sm.ols(formula='Sales ~ TV + Radio',
               data=df).fit()
print(model.summary())

Como esperado, não houve impacto na qualidade do modelo.

## Exemplo Auto MPG

Vamos agora aplicar a Regressão Linear Múltipla em um novo conjunto de dados, chamado Auto MPG. Esse conjunto de dados pode ser extraído diretamente da biblioteca *seaborn*.

Este conjunto de dados descreve o consumo de diferentes carros (medido pelo número de milhas percorridas com o uso de um galão de gasolina) com base no seguinte conjunto de características de tais carros:

- cylinders -> Número de cilíndros.

- displacement -> Capacidade do motor.

- horsepower -> Potência (cavalo-vapor).

- weight -> Peso.

- acceleration -> Tempo, em segundos, até atingir a velocidade de 100 km/h (partindo do carro em repouso).

- model_year -> Ano do modelo do carro

- origin -> Local de produção do carro.

- name -> Nome do modelo do carro.

Uma característica interessante nesse dado é que ele possui variáveis categóricas, o que traz uma interpretação diferente para o coeficiente associado a elas.

In [ ]:
import seaborn as sns
df = sns.load_dataset('mpg')
df.head()


Primeriamente, vamos observar a dispersão dos dados e os coeficientes de correlação.

In [ ]:
sns.pairplot(df,
             x_vars= df.drop('mpg', axis=1).columns,
             y_vars='mpg',
             kind='scatter')
plt.show()

In [ ]:
# Correlação linear

df.drop(columns=['origin','name']).corr()[["mpg"]]

Note que há correlações fortes entre *mpg* e a maioria das variáveis (numéricas) preditivas.

Vamos agora aplicar a Regressão Linear Múltipla. Para as análises, vamos considerar apenas os atributos *horsepower*, *weight*, *acceleration* e *origin* como variáveis preditoras.

In [ ]:
# Fit the resgression line using 'OLS'

model = sm.ols(formula='mpg ~ horsepower + weight + acceleration + origin',
               data=df).fit()
print(model.summary())

O modelo de regressão obtido foi o seguinte:

$$\widehat{y} = 44.2680 + 1.7759origin[T.japan] - 0.9766origin[T.usa] - 0.0564horsepower - 0.0048weight - 0.0309acceleration, $$

em que $\widehat{y}$ é o `mpg` esperado para certas características do carro.

No modelo de regressão, é comum deixar uma categoria como referência. Assim, a(s) outra(s) categorias mensuram o efeito de passar da categoria de referência para outra categoria.

**Em nosso exemplo:**

A variável `origin` tem três categorias: *europe*, *japan* e *usa*. Note que as variáveis que estão no modelo são *origin[T.japan]* e *origin[T.usa]*, assim, elas estão medindo como se altera a variável `origin` quando passamos de considerar o local de produção do veículo na Europa para Japão ou USA. Observe que *origin[T.japan]* tem sinal positivo, indicando que a `mpg` de veículos cuja produção é no Japão é maior quando comparada com a produção na Europa. Por outro lado, *origin[T.usa]* tem sinal negativo, indicando que a `mpg` de veículos cuja produção é nos USA é menor quando comparada com a produção na Europa.

Note também que a variável *origin[T.usa]* não é significante (valor-p > 0.05).

Na prática, quando vamos calcular `mpg` usando o modelo acima, substituimos *origin[T.japan]* por 1 se a produção foi no Japão ou zero, caso contrário. O mesmo raciocínio vale para *origin[T.usa]*.

# **Exercício: Verifiquem como fica o modelo ao transformar a variável *origin* em dummies**

In [ ]:
# Valores preditos pelo modelo

df['predicted'] = model.predict(df[["horsepower", "weight", "acceleration", "origin"]])
df.head()

Usando o modelo de regressão linear ajustado, qual é o valor esperado de `mpg` para veículos com as seguintes características:
- horsepower: 140
- weight: 3450
- acceleration: 11.5
- origin: japan

In [ ]:
# Prevendo para novos valores (não estão na base de dados)

new_value=pd.DataFrame({'horsepower': [140],
                        'weight': 3450,
                        'acceleration': 11.5,
                        'origin': "japan"})

model.predict(new_value)

In [ ]:
# Resíduos

residuals=model.resid
df["res"]=residuals

In [ ]:
# Gráfico de dispersão dos resíduos

plt.figure(figsize=(10, 3))

plt.subplot(1, 2, 1)
plt.scatter(df["predicted"], df["res"])
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Valores ajustados')
plt.ylabel('Resíduos')
plt.title('Gráfico de Dispersão dos Resíduos')

plt.subplot(1, 2, 2)
plt.hist(residuals)

plt.tight_layout()
plt.show()

Esses gráficos sugerem que os resíduos se desenvolvem aleatóriamente em torno de zero e seguem aproximadamente uma distribuição Normal.

In [ ]:
# Analisando possíveis outliers (fora de 2.5 vezes o desvio-padrão em relação à média)

limite = 2.5*df.res.std()
df[(df.res<-limite) | (df.res>limite)].sort_values(by=['res'])

In [ ]:
import matplotlib.pyplot as plt

std_error_dict = model.bse.to_dict() # Extrai os nomes dos atributos e std err e coloca em um dicionário
coeff_dict = model.params.to_dict() # Extrai os nomes dos atributos e coeficientes e coloca em um dicionário
t_names = []
t_val = []
for key in std_error_dict: # Separa os nomes e valores do dicionário
  if key != 'Intercept':
    t_names.append(key)
    t_val.append(abs(coeff_dict[key]/std_error_dict[key]))

plt.pie(t_val, labels=t_names, autopct='%0.2f%%') # Gráfico de pizza

# **Vantagens e desvantagens da Regressão Linear**

**Vantagens**

- O modelo é de fácil implementação e interpretação.

- Não necessita um esforço computacional grande, uma vez que a criação do modelo é obtida de forma analítica.

- Se as suposições sobre o problema são atendidades, pode resultar em boas predições.

**Desvantagens**

- Caso as suposições não sejam atendidas, o modelo não fornecerá bons resultados.

- Consegue modelar, apenas, uma relação linear entre as variáveis de entrada e a variável de saída. Sendo assim, pode resultar em um *underfitting*, não conseguindo estabeler uma relação complexa entre os dados.

- Como não leva em consideração interações entre variáveis, a interpretação é sensível à multicolinearidade, ou seja, à presença de variáveis correlacionadas.